# Demostración

In [1]:
import pandas as pd
import numpy as np
import pickle
import time
import datetime
import re

import sys

from tensorflow import keras

In [2]:
model = keras.models.load_model("modelo_pronosticador_regina.h5")

file = open("norm.params", "rb")
train_mean, train_std, wind_dic = pickle.load(file)
file.close()

In [3]:
drop = [0, 1, 3, 4, 12, 13, 15, 14, 9, 20, 21, 27, 31, 32]

def transform_date(date, date_format):
    date = pd.Timestamp.timestamp(pd.Timestamp(datetime.datetime.strptime(date, date_format)))
    
    day = 24*60*60
    year = 365.2425 * day
    
    day_sin  = np.sin(date * (2 * np.pi / day))
    day_cos  = np.cos(date * (2 * np.pi / day))
    year_sin = np.sin(date * (2 * np.pi / year))
    year_cos = np.cos(date * (2 * np.pi / year))
    
    return [day_sin, day_cos, year_sin, year_cos]

def convert(data, regex = "\s+(?!$)", date_format = "%d/%m/%y %H:%M"):
    data = data.splitlines()
    conv_data = np.zeros((0, 23))
    
    check = len(data) - 90
    if not check==0:
        print("Hay ", check, " muestras de más.") if check > 0 else print("Faltan ", abs(check), " muestras.")
        return
    
    for sample in data:
        sample = re.split(regex, sample)
        date = sample[0] + " " + sample[1]
        sample[8]  = wind_dic[sample[8]]
        sample[11] = wind_dic[sample[11]]
        sample = np.delete(sample, drop)
        sample = sample.astype(float)
        sample[0] -= 2
        
        sample = np.concatenate((sample, transform_date(date, date_format)))
        
        sample = (sample-train_mean)/train_std

        conv_data = np.vstack((conv_data, sample))
    
    conv_data = np.reshape(conv_data, (1, 90, 23))
    return conv_data

## Helada 27/10/21 2:50 AM
### Predicción a las 21:00

### Predicción a las 22:00

### Predicción a las 23:00

## Helada 5/10/21 2:40 AM
### Predicción a las 21:00

### Predicción a las 22:00

### Predicción a las 23:00

### Predicción a las 00:00